<a href="https://colab.research.google.com/github/sihyeon3523/Six_of_cells/blob/modeling/Seunguk_(3)_lyricModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

# 0. Install & import libraries 

In [ ]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2 
print(tensorflow.__version__) # 2.8.0  
print(transformers.__version__) # 4.8.2  
print(tensorflow_addons.__version__) # 0.16.1 

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm
import ast  # str list to list

1.0.2
2.8.0
4.8.2
0.17.0


In [ ]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# Import Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.getcwd()

'/content/drive/My Drive/likelion/proj_fin_seunguk'

In [ ]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/likelion/proj_fin_seunguk'# google drive project path
data_path = 'colab_data/temp_data/' # model data path
checkpoint_path = 'colab_data/temp_data/saved_models/' # saved model path

os.chdir(path)

# 1. Load the Data

In [ ]:
df_allsongs_original = pd.read_csv('df_allsongs.csv', lineterminator='\n')

In [ ]:
df_allsongs_original.head(3)

,Unnamed: 0,Title,Artist,Date,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."


In [ ]:
df_allsongs = df_allsongs_original[["Title", "Artist", "Date", "Genre", "preprocess_Lyric_ver3"]]
df_allsongs

,Title,Artist,Date,Genre,preprocess_Lyric_ver3
0,나의 X에게,경서,2022.04.24,발라드,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,취중고백,김민석 (멜로망스),2021.12.19,발라드,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마","['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."
3,듣고 싶을까,MSG워너비(M.O.M),2022.02.26,발라드,"['우리 함께 듣던 그 노랠 듣고 싶을까', '듣고 나서 잠시 날 생각은 할까', ..."
4,다정히 내 이름을 부르면,경서예지,2021.05.19,발라드,"['끝없이 별빛이 내리던 밤', '기분 좋은 바람이', '두 빰을 스치고', '새벽..."
...,...,...,...,...,...
3604,애수,진시몬,2004.10.01,성인가요/트로트,"['아직도 모르겠어 난 정말 ', '', '꿈이라 생각해야 하는지 ', '', '너..."
3605,단장의 미아리 고개,송가인,2020.12.26,성인가요/트로트,"['미아리 눈물고개', '임이 넘던 이별 고개', '화약 연기 앞을 가려', '눈 ..."
3606,어머님 사랑합니다,송가인,2021.05.27,성인가요/트로트,"['꽃다운 열아홉에', '연지 찍고 시집와서', '손발이 터지도록', '고생하신 어..."
3607,영동 부르스,송가인,2021.06.13,성인가요/트로트,"['헤어지기 싫어서 애태우던 그 날 밤', '피눈물에 얼룩진 그대의 모습', '어차..."


In [ ]:
print(df_allsongs.info())
print(df_allsongs.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3609 entries, 0 to 3608
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Title                  3609 non-null   object
 1   Artist                 3609 non-null   object
 2   Date                   3609 non-null   object
 3   Genre                  3609 non-null   object
 4   preprocess_Lyric_ver3  3609 non-null   object
dtypes: object(5)
memory usage: 141.1+ KB
None
       Title Artist        Date Genre  \
count   3609   3609        3609  3609   
unique  3358    947        2042    41   
top       고백    임영웅  2022.05.20    댄스   
freq      10     90          19   486   

                                    preprocess_Lyric_ver3  
count                                                3609  
unique                                               3609  
top     ['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ...  
freq                                        

In [ ]:
df_allsongs.preprocess_Lyric_ver3.iloc[12]

"['느낌이 오잖아 ', '떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '어서 내 손을 잡아', '우연히 고개를 돌릴 때 마다', '눈이 마주치는 건', '며칠 밤 내내 꿈속에 나타나', '밤새 나를 괴롭히는 건', '그 많은 빈자리 중에서 하필', '내 옆자릴 고르는 건', '나도 모르게 어느새 실없는 웃음', '흘리고 있다는 건', '그럼 말 다했지 뭐', '우리 얘기 좀 할까', '느낌이 오잖아 떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '핸드폰 진동에 ', '심장이 덜컥내려 앉는다는 건', '오 나도 모르게 ', '어느새 짓궂은 네 말투', '자꾸 듣고 싶은걸', '어떡해', '저기 멀리 걸어온다', '눈이 마주친다', '언제까지 넌 모른척 할거니', '사랑이 온거야 너와 나 말이야', '네가 좋아 정말 못 견딜 만큼', '그거면 된거야 ', '더는 생각하지 말란 말이야', '네 맘 가는 그대로', '느낌이 오잖아 떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '그냥 내 손을 잡아', '지금 내 손을 잡아', ' ', '']"

# Colab 끊겨서 뒷부분 장르 태깅 안되는 경우 방지 all rows shuffle

In [ ]:
df_allsongs = df_allsongs.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset

# Load saved Model


In [ ]:
# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver) 
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

# Classifier as a function

In [ ]:
def predict_sentiment_element(sentence, tokenizer, model):
    """
    가사 내 문장 별 확률 값 추출
    """    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence), # 특수문자 제거
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    pred = model.predict(new_inputs) 
    all_pred_proba = np.round(pred * 100, 2)
    
    #top_pred_proba = np.round(np.max(pred) * 100, 2) 
    #pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(pred, axis=1)[0]] # 수정하면 4개 클래스 다 나올듯! 
    #print("{}% 확률로 {} 텍스트입니다.".format(top_pred_proba, pred_class))

    return all_pred_proba 

In [ ]:
def predict_sentiment_lyrics(lyrics):
    """
    가사의 감성 확률 값 추출, 문장별 감성 확률의 정규화 평균  
    가사내 문장 단위로 predict_sentiment_element 함수 적용   
    마지막 원소 '' 빈칸임. <br/> 태그 기준으로 split해서 나옴, 빈칸인 element 제거
    """
    lyrics = ast.literal_eval(lyrics) # str list to list
    lyrics = list(filter(lambda x: x != '', lyrics))

    print(len(lyrics)) 
    print(lyrics)


    for i in range(len(lyrics)):
        
        if lyrics[i] != '':
            all_pred_proba = predict_sentiment_element(lyrics[i], tokenizer, model)
            anger = all_pred_proba[0][0]
            scary = all_pred_proba[0][1]
            sad = all_pred_proba[0][2]
            happy = all_pred_proba[0][3]
            #print(i, lyrics[i], all_pred_proba) 

        anger += anger 
        scary += scary
        sad  += sad
        happy += happy

    total_array = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)
    top_pred_prob = total_array[0][np.argmax(total_array, axis=1)[0]]
    top_pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(total_array, axis=1)[0]]

    return total_array, top_pred_class, top_pred_prob

In [ ]:
# test_list = df_allsongs.preprocess_Lyric_ver3.iloc[35]
# total_array, top_pred_class, top_pred_prob = predict_sentiment_lyrics(test_list)

In [ ]:
# print(total_array) 
# print(top_pred_class)  
# print(top_pred_prob)  

preprocess_Lyric_ver3에 적용 <br> 

list가 str 형태로 row에 저장되었기 때문에 ast library로 다시 list로 바꿔주어야한다.<br>
(str list to list)

# Apply model

In [ ]:
# Emotion = []
# Probability = []
# 분노혐오 = []
# 놀람공포 = []
# 슬픔 = []
# 행복 = []


# for row in tqdm(df_allsongs.iterrows()):


#         total_array, top_pred_class, top_pred_prob = predict_sentiment_lyrics(row[1][-1])
#         Emotion.append(top_pred_class)
#         Probability.append(top_pred_prob)
#         분노혐오.append(total_array[0][0])
#         놀람공포.append(total_array[0][1])
#         슬픔.append(total_array[0][2])
#         행복.append(total_array[0][3]) 

# df_allsongs['Emotion'] = Emotion
# df_allsongs['Proba'] = Probability
# df_allsongs['분노혐오'] = 분노혐오
# df_allsongs['놀람공포'] = 놀람공포
# df_allsongs['슬픔'] = 슬픔
# df_allsongs['행복'] = 행복

# df_allsongs.to_csv('songtagged_may27.csv')

In [ ]:
df_allsongs['Emotion'] = " "
df_allsongs['Probability'] = " "
df_allsongs['분노혐오'] = " "
df_allsongs['놀람공포'] = " "
df_allsongs['슬픔'] = " "
df_allsongs['행복'] = " "

for i in tqdm(df_allsongs.index):
    lyrics = df_allsongs['preprocess_Lyric_ver3'].iloc[i]
    total_array, top_pred_class, top_pred_prob = predict_sentiment_lyrics(lyrics)
    df_allsongs['Emotion'].iloc[i] = top_pred_class 
    df_allsongs['Probability'].iloc[i] = top_pred_prob
    df_allsongs['분노혐오'].iloc[i] = total_array[0][0]
    df_allsongs['놀람공포'].iloc[i] = total_array[0][1]
    df_allsongs['슬픔'].iloc[i] = total_array[0][2]
    df_allsongs['행복'].iloc[i] = total_array[0][3]

    df_allsongs.to_csv('songtagged_may29.csv')

  0%|          | 0/3609 [00:00<?, ?it/s]

97
["I can't understand what people are sayin'", '어느 장단에 맞춰야 될지', '한 발자국 떼면 한 발자국 커지는 shadow', '잠에서 눈을 뜬 여긴 또 어디', '어쩜 서울 또 New York or Paris', '일어나니 휘청이는 몸', 'Look at my feet, look down', '날 닮은 그림자', '흔들리는 건 이놈인가', '아니면 내 작은 발끝인가', '두렵잖을 리 없잖아', '다 괜찮을 리 없잖아', '그래도 I know', '서툴게 I flow', '저 까만 바람과 함께 날아', 'Hey na na na', '미치지 않으려면 미쳐야 해', 'Hey na na na', '나를 다 던져 이 두 쪽 세상에', 'Hey na na na', 'Can’t hold me down cuz you know I’m a fighter', '제 발로 들어온 아름다운 감옥', "Find me and I'm gonna live with ya", '(Eh-oh)', '가져와 bring the pain oh yeah', '(Eh-oh)', '올라타봐 bring the pain oh yeah', "Rain be pourin'", "Sky keep fallin'", 'Everyday oh na na na', '(Eh-oh)', '가져와 bring the pain oh yeah', 'Bring the pain', '모두 내 피와 살이 되겠지', 'Bring the pain', 'No fear, 방법을 알겠으니', '작은 것에 breathe', '그건 어둠 속 내 산소와 빛', '내가 나이게 하는 것들의 힘', '넘어져도 다시 일어나 scream', '넘어져도 다시 일어나 scream', '언제나 우린 그랬으니', '설령 내 무릎이 땅에 닿을지언정', '파묻히지 않는 이상', '그저 그런 해프닝쯤 될 거란 걸', 'Win no matter what', 'Win no matter what', 'Win no matter what', 

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]
  0%|          | 1/3609 [01:26<86:48:56, 86.62s/it]

35
['함께 했던 많은 계절이', '봄에 눈이 녹듯 사라진다 해도', '아직 나는 너를 기억해', '세상 무엇보다 빛나던 모습을', '어떤 날도 어떤 말도', '우리 안녕이라 했었던 그 날도', '저기 어딘가에 꿈을 꾸던 시간조차도', '오랜 영화처럼 다시', '빛이 되어 남을테니', '어떤 날도 어떤 말도', '우리 안녕이라 했었던 그 날도', '문득 고갤 드는', '가슴 아픈 기억조차도', '언젠가는 아름다운', '눈물들로 남을테니까', '아직 나는 너를 기억해', '세상 무엇보다 빛나던 모습을', '어떤 날도 어떤 말도', '우리 안녕이라 했었던 그 날도', '저기 어딘가에 꿈을 꾸던 시간조차도', '오랜 영화처럼 다시', '빛이 되어 남을테니', '저기 우리 행복했었던 시간이', '아직도 손에 잡힐 듯', '어딘가 아쉬운 마음도', '때로는 서운한 마음도', '언젠가 함께 했었던 그 약속도', '어떤 날도 어떤 말도', '우리 안녕이라 했었던 그 날도', '문득 고갤 드는', '가슴 아픈 기억조차도', '언젠가는 아름다운', '눈물들로 남을테니까', '함께 했던 많은 계절은', '비록 여기에서 끝이 난다해도']


  0%|          | 2/3609 [01:45<47:08:41, 47.05s/it]

...

40
['사소한 것들 땜에', '크게 다퉜었지', '하루 종일 말도 안 하고', '꿍해 있었지', '사랑이었나 봐', '그땐 몰랐지만', '전부 다 지나고 나서', '깨달았어', '둘 중 하나 누구 한 명만', '한 번만 져줬었다면 우린', '어떻게 됐을까', '연락을 늦게 해서', '크게 다퉜었지', '나 일하느라 그랬어', '하루 종일 혼났지', '사랑이었나 봐', '그땐 몰랐지만', '전부 다 지나고 나서', '깨달았어', '둘 중 하나 누구 한 명만', '한 번만 져줬었다면 우린', '어떻게 됐을까', '가끔은 연락을 안 하고 계속 잤다고', '아 미안 영화 보느라고 못 봤다고', '그 영화 나랑 안 보고', '왜 혼자 봤냐고', '아 미안 친구가 갑자기 약속 잡았어', '화나면 우린 하루 종일 말도 안 하고', '될 대로 되란 식 친구와 술을 마시고', '우리 싸우는 거 보면', '가끔은 올림픽 같았어', '1등이 아니면 성이 안 차도', '그건 그래도', '사랑이었나 봐', '그땐 몰랐지만', '전부 다 지나고 나서', '깨달았어', '둘 중 하나 누구 한 명만', '한 번만 져줬었다면 우린', '어떻게 됐을까']


  0%|          | 4/3609 [02:26<29:23:43, 29.35s/it]

52
['서울은 좁아서', '어딜 가도 네가 따라와', '다시 돌아온 이 계절처럼', '또 너를 떠올려', '잘 해내고 있다 믿었지만', '넌 내게 불어와', '다시 흔들린 내 마음처럼', '살랑거리는 건 봄이야', '살랑거리는 건 봄이야', '새로운 사랑으로 널 잊으래', '근데 난 그게 진짜 안돼', '압구정 사거리 단둘이 걷던 길', '거기서부터 시작된', '우리 사랑 일기 첫 페이지 날짜가', '너처럼 따뜻한 봄이야', '근데 이젠 남이야', '눈치 없는 봄바람만 괜시리', '나의 맘속에 불어', '서울은 좁아서', '어딜 가도 네가 따라와', '다시 돌아온 이 계절처럼', '또 너를 떠올려', '잘 해내고 있다 믿었지만', '넌 내게 불어와', '다시 흔들린 내 마음처럼', '살랑거리는 건 봄이야', '되돌릴 수 없는 과거처럼', '차가운 겨울이 녹아내려', '왜 나만 괴롭고 또 외로워 난 진짜', '벌거벗어버린 것만 같아', '나만 몰랐어 언제나 바보야', '네가 싫어하던 내가 미안하단 말', '입버릇처럼 나와 이제 없지만', '봄바람 타고서 전화를 걸어', '부재중 날 피해도 괜찮아', '이거 나도', '실수인 거 알면서 그냥 하는 거야', '이러지 않으면 잘못될 것만 같아', '마음껏 비웃어 나를', '아마도 난 너의 술자리 안주', '알잖아 I just want you back', '가만히 두지를 않네', '서울은 좁아서', '어딜 가도 네가 따라와', '다시 돌아온 이 계절처럼', '또 너를 떠올려', '잘 해내고 있다 믿었지만', '넌 내게 불어와', '다시 흔들린 내 마음처럼', '살랑거리는 건 봄이야', '살랑거리는 건 봄이야', '살랑거리는 건']


  0%|          | 5/3609 [02:55<29:07:23, 29.09s/it]

36
['우리 이별을 말 한지 ', '겨우 하루 밖에 지나지 않았어', '하지만 너무 이상하게도 ', '내 맘은 편안해 자유로운 기분', '이틀 이틀째가 되던 날', '온 몸이 풀리고 가슴이 답답해', '삼 일째 되던 날', '내 심장 소리가 너무 커서 잠도 못 자', '나흘 되던 날', '눈 앞이 캄캄해지고', '오일 되던 날', '눈물만 주루룩 엿새 되던 날', '가슴이 너무 허전해', '하루 온 종일 먹기만 하네요', '일주일 일주일이 되던 날', '노래 속 가사가 모두 내 얘기 같고', '드라마 영화 속에 나오는 ', '삼류 사랑 얘기가 모두 ', '다 내 얘기만 같아', '한 달 한 달이 되던 날', '니가 좋아 했던 긴 머릴 자르고', '니가 싫어했었던 야한 옷을 입으니 ', '이별이 이제서 야 실감나네', '일 년 되던 날', '널 많이 닮은 사람과 사랑에 빠져 ', '행복을 찾았고', '가끔은 너의 소식에 ', '조금은 신경 쓰여도', '그냥 뒤돌아 웃음 짓게 되네', '사랑이 그런건 가 봐 ', '세월이 약이였나봐', '그 때는 정말 죽을 것 같았어', '하지만 지금 사랑이 ', '또 다시 아픔을 줘도', '나는 웃으며 이별을 맞을래 ', '사랑은 또 다시 올테니까']
